In [1]:
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk
!pip install rouge-score
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 50.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 62.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=047545551a4512ab44351ce88f126c279bb477cc1de013569fc2ef761efc0e2c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [41]:
import logging
import pandas as pd
import nltk
import tqdm
import json
import datasets
import numpy as np
import tensorflow as tf
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from datasets import Dataset
from itertools import chain
from tensorflow import keras
sns.set()

In [3]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 1024  # Maximum length of the input to the model
MIN_TARGET_LENGTH = 5  # Minimum length of the output by the model
MAX_TARGET_LENGTH = 128  # Maximum length of the output by the model
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 1e-6  # Learning-rate for training our model
# LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# This notebook is built on the t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "t5-small"

In [4]:
data = []

for x in range(1, 6):
    with open(f"/kaggle/input/indosum/indosum/train.0{str(x)}.jsonl", 'r') as f:
        json_list_2 = list(f)
        for json_str_2 in json_list_2:
            a = json.loads(json_str_2)
            data.append(a)

In [5]:
len(data)

71353

In [6]:
data[0].keys(), len(data)

(dict_keys(['category', 'gold_labels', 'id', 'paragraphs', 'source', 'source_url', 'summary']),
 71353)

In [7]:
data_category = []

for x in range(0, 5):
    data_category.append(data[x]["category"])
data_category

['tajuk utama', 'teknologi', 'hiburan', 'tajuk utama', 'tajuk utama']

In [8]:
def paragraphs_to_text(raw_paragraph_list):
    new_paragraph_list = []
    for i, paragraph in enumerate(raw_paragraph_list):
        paragraph_list = []
        for sentence in paragraph:
            sentence = ' '.join(sentence)
            paragraph_list.append(sentence)
        
        new_paragraph = ' '.join(paragraph_list)
        new_paragraph_list.append(new_paragraph)

    paragraph_str = ' '.join(new_paragraph_list)
    return paragraph_str

paragraphs_to_text(data[0]['paragraphs'])

'Jakarta , CNN Indonesia - - Dokter Ryan Thamrin , yang terkenal lewat acara Dokter Oz Indonesia , meninggal dunia pada Jumat ( 4 / 8 ) dini hari . Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu . Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia . Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat . " Setahu saya dia orangnya sehat , tapi tahun lalu saya dengar dia sakit . ( Karena ) sakitnya , ia langsung pulang ke Pekanbaru , jadi kami yang mau jenguk juga susah . Barangkali mau istirahat , ya betul juga , kalau di Jakarta susah isirahatnya , " kata Lula kepada CNNIndonesia.com , Jumat ( 4 / 8 ) . Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh . Dia juga tak tahu penyakit apa yang diderita Ry

In [9]:
paragraphs_list = []

# Menggunakan data_2 
for x in range(0,len(data)):
    paragraphs_list.append(paragraphs_to_text(data[x]["paragraphs"]))
paragraphs_list[:5]

['Jakarta , CNN Indonesia - - Dokter Ryan Thamrin , yang terkenal lewat acara Dokter Oz Indonesia , meninggal dunia pada Jumat ( 4 / 8 ) dini hari . Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu . Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia . Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat . " Setahu saya dia orangnya sehat , tapi tahun lalu saya dengar dia sakit . ( Karena ) sakitnya , ia langsung pulang ke Pekanbaru , jadi kami yang mau jenguk juga susah . Barangkali mau istirahat , ya betul juga , kalau di Jakarta susah isirahatnya , " kata Lula kepada CNNIndonesia.com , Jumat ( 4 / 8 ) . Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh . Dia juga tak tahu penyakit apa yang diderita R

In [10]:
def summary_to_text(summary):
    new_summary = []
    for x in summary:
        new_summary.append(' '.join(x))
    hasil = ' '.join(new_summary)
    return hasil

summary_to_text(data[0]["summary"])

'Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan Thamrin menyebut kawannya itu sudah sakit sejak setahun yang lalu . Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia . Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat .'

In [11]:
# summary_list = []
# summary = [summary_list.append(summary_to_text(data[x]["summary"])) for x in range(0, len(data))]
# summary_list[:5]

summary_list = []

for x in range(0, len(data)):
    summary_list.append(summary_to_text(data[x]["summary"]))
summary_list[:5]

['Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan Thamrin menyebut kawannya itu sudah sakit sejak setahun yang lalu . Lula menuturkan , sakit itu membuat Ryan mesti vakum dari semua kegiatannya , termasuk menjadi pembawa acara Dokter Oz Indonesia . Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru , Riau untuk menjalani istirahat .',
 'Asus memperkenalkan \xa0 ZenFone generasi keempat dan keduanya sama-sama dibekali setup kamera ganda di depan . Mereka adalah Asus ZenFone 4 Selfie Pro ZD552KL dan ZenFone 4 Selfie ZD553KL . Dua ZenFone Selfie anyar ini kabarnya diracik sedemikian rupa sebagai jawaban atas kekurangan yang ada di perangkat - perangkat kompetitor , khususnya pada aspek jangkauan lensa dan performa di kondisi low-light .',
 'Dinas Pariwisata Provinsi Bengkulu kembali menggelar kegiatan Bimbingan Teknis ( Bimtek ) SDM Kepariwisataan dalam menyongson " Visit 2020 Wonderful Bengkulu " pada 8 - 10 November 2017 yang lalu . Kegiatan 

In [12]:
data = pd.DataFrame({
    "text":paragraphs_list,
    "summary":summary_list
})
data

,text,summary
0,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
1,Selfie ialah salah satu tema terpanas di kalan...,Asus memperkenalkan ZenFone generasi keempat...
2,"Jakarta , CNN Indonesia - - Dinas Pariwisata P...",Dinas Pariwisata Provinsi Bengkulu kembali men...
3,Merdeka.com - Indonesia Corruption Watch ( ICW...,Indonesia Corruption Watch ( ICW ) meminta Kom...
4,Merdeka.com - Presiden Joko Widodo ( Jokowi ) ...,Jokowi memimpin upacara penurunan bendera . Us...
...,...,...
71348,Merdeka.com - Direktorat Reserse Kriminal Khus...,Direktorat Reserse Kriminal Khususnya Polda Ri...
71349,Merdeka.com - Kementerian Perdagangan telah me...,Kementerian Perdagangan telah menetapkan Harga...
71350,Rimanews - Menteri Perhubungan Budi Karya Suma...,Menteri Perhubungan Budi Karya Sumadi akan mem...
71351,JUARA.net - Duel antara Floyd Mayweather dan C...,Sejumlah persipan jelang duel antara Floyd May...


In [13]:
data_hasil = pd.read_csv("/kaggle/input/summary-ind-2/hasil.csv")
data_hasil

,text,summary
0,Evakuasi warga negara Indonesia dari Timor Le...,Evakuasi warga negara Indonesia dari Timor Les...
1,Pusat Vulkanologi dan Mitigasi Bencana Geolog...,Status Gunung Batutara ditetapkan siaga tiga k...
2,Harga sejumlah kebutuhan pokok di berbagai dae...,"Harga bahan pokok seperti minyak goreng, telur..."
3,Romli Atmasasmita dituntut lima tahun penjara ...,Jaksa menilai Romli Atmasasmita menyalahgunaka...
4,Puluhan hektare lahan hutan lindung di kawasan...,Puluhan hektare lahan hutan lindung di kawasan...
...,...,...
143436,Sarung tenun yang dibuat dengan alat tenun buk...,"Sarung tenun ATBM yang diproduksi di Pemalang,..."
143437,"Laila Ali, putri legenda tinju Muhammad Ali, ...",Putri legenda tinju Muhammad Ali melahirkan ba...
143438,"Inspeksi mendadak Dinas Kesehatan Magelang, J...","Dinas Kesehatan Magelang, Jateng, menemukan ka..."
143439,Ujian hidup tengah dialami Nuri Maulida. Saat ...,Ayah Nuri Maulida saat ini tengah terbaring di...


In [14]:
data = data.merge(data_hasil, how="outer")
data

,text,summary
0,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
1,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
2,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
3,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
4,Selfie ialah salah satu tema terpanas di kalan...,Asus memperkenalkan ZenFone generasi keempat...
...,...,...
214789,Sarung tenun yang dibuat dengan alat tenun buk...,"Sarung tenun ATBM yang diproduksi di Pemalang,..."
214790,"Laila Ali, putri legenda tinju Muhammad Ali, ...",Putri legenda tinju Muhammad Ali melahirkan ba...
214791,"Inspeksi mendadak Dinas Kesehatan Magelang, J...","Dinas Kesehatan Magelang, Jateng, menemukan ka..."
214792,Ujian hidup tengah dialami Nuri Maulida. Saat ...,Ayah Nuri Maulida saat ini tengah terbaring di...


In [15]:
pola = r"^\n{2,}|^\r\n{2,}"
print("Jumlahnya =", data["text"].str.contains(pola).sum())

Jumlahnya = 16


In [16]:
data = data[data["text"].str.contains(pola) == False]
data

,text,summary
0,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
1,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
2,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
3,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...
4,Selfie ialah salah satu tema terpanas di kalan...,Asus memperkenalkan ZenFone generasi keempat...
...,...,...
214789,Sarung tenun yang dibuat dengan alat tenun buk...,"Sarung tenun ATBM yang diproduksi di Pemalang,..."
214790,"Laila Ali, putri legenda tinju Muhammad Ali, ...",Putri legenda tinju Muhammad Ali melahirkan ba...
214791,"Inspeksi mendadak Dinas Kesehatan Magelang, J...","Dinas Kesehatan Magelang, Jateng, menemukan ka..."
214792,Ujian hidup tengah dialami Nuri Maulida. Saat ...,Ayah Nuri Maulida saat ini tengah terbaring di...


In [17]:
train_list, test_list = train_test_split(data, test_size=0.2)
print(f"Jumlah train {len(train_list)}", f"Jumlah test {len(test_list)}")

Jumlah train 171347 Jumlah test 42837


In [18]:
train_list = train_list.astype("str")
test_list = test_list.astype("str")

In [19]:
train = dict(train_list)
test = dict(test_list)

In [20]:
# Ubah kedalam tipe DatasetDict()
train = Dataset.from_dict(train)
test = Dataset.from_dict(test)
data = datasets.DatasetDict({"train":train, "test":test})
data

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 171347
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 42837
    })
})

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [22]:
# tokenizer("Hey kamu lagi ngapain?") 
tokenizer(["Kamu kemarin kemana aja?", "Kenapa tidak datang tadi malam?"])

{'input_ids': [[8329, 76, 3, 1050, 12181, 3, 1050, 348, 9, 3, 9, 1191, 58, 1], [5704, 9750, 3, 17, 23, 26, 1639, 331, 1725, 3, 17, 9, 26, 23, 1460, 265, 58, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [23]:
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [24]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [25]:
tokenized_datasets = data.map(preprocess_function, batched=True)
tokenized_datasets

  0%|          | 0/172 [00:00<?, ?ba/s]

  0%|          | 0/43 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 171347
    })
    test: Dataset({
        features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 42837
    })
})

In [26]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [27]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
data_collator

DataCollatorForSeq2Seq(tokenizer=PreTrainedTokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<ext

In [28]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(124)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [30]:
import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # We will print only the F1 score, you can use other aggregation metrics as well
    result = {"RougeL": result["f1_score"]}

    return result

In [31]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

# For now we will use our test set as our validation_data
history = model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

21418/21418 [==============================] - 11441s 532ms/step - loss: 1.6616 - val_loss: 1.4471 - RougeL: 0.1667


In [72]:
kalimat_prediksi = "Pemerintah telah menetapkan urusan kebudayaan menjadi urusan wajib. Artinya, negara harus hadir dalam upaya-upaya pembangunan di bidang kebudayaan. Melalui fasilitas ini diharapkan komunitas budaya dan desa adat selalu terjaga keberadaannya.Keberadaan masyarakat adat dan komunitas budaya menjadi media strategi bagi upaya interaksi masyarakat pendukungnya, sosialisasi, dan pendidikan nilai-nilai budaya kepada masyarakat luas."

In [73]:
kalimat_prediksi

'Pemerintah telah menetapkan urusan kebudayaan menjadi urusan wajib. Artinya, negara harus hadir dalam upaya-upaya pembangunan di bidang kebudayaan. Melalui fasilitas ini diharapkan komunitas budaya dan desa adat selalu terjaga keberadaannya.Keberadaan masyarakat adat dan komunitas budaya menjadi media strategi bagi upaya interaksi masyarakat pendukungnya, sosialisasi, dan pendidikan nilai-nilai budaya kepada masyarakat luas.'

In [74]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summarizer(
    kalimat_prediksi,
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)

RuntimeError: Impossible to instantiate a pipeline with tokenizer specified but not the model as the provided tokenizer may not be compatible with the default model. Please provide a PreTrainedModel class or a path/identifier to a pretrained model when providing tokenizer.

In [36]:
# model.push_to_hub("transformers-qa", organization="keras-io")
# tokenizer.push_to_hub("transformers-qa", organization="keras-io")

In [37]:
# from transformers import TFAutoModelForSeq2SeqLM

# model = TFAutoModelForSeq2SeqLM.from_pretrained("your-username/my-awesome-model"

In [38]:
# model.save_weights("model_summary_ind.h5")

LOAD MODEL

In [64]:
import keras
import argparse
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq


MODEL_CHECKPOINT = "t5-small"
LEARNING_RATE = 1e-7
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 256

def build_model():
    if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
        prefix = "summarize: "
    else:
        prefix = ""

    model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer)
    return model

In [65]:
model = build_model()
model = model.load_weights('model_summary_ind.h5')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [66]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [67]:
kalimat_prediksi = "Ayam termasuk kelompok hewan ovipar. Artinya ayam berkembangbiak dengan cara bertelur. Ayam betina umumnya mulai bertelur setelah berusia 5 bulan. Jumlah telur yang dihasilkan ayam bervariasi tergantung jenis ayam, usia, dan kondisi lingkungan tempat tinggal ayam tersebut."

In [68]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summarizer(
    kalimat_prediksi,
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)

RuntimeError: Impossible to instantiate a pipeline with tokenizer specified but not the model as the provided tokenizer may not be compatible with the default model. Please provide a PreTrainedModel class or a path/identifier to a pretrained model when providing tokenizer.